In [2]:
import numpy as np
import pandas as pd

In [9]:
 # bus_data format: {node ID, node type(1=PQ,2=PV,3=SLACK), conductance of shunt adm (Gsh,pu), 
# susceptance of shunt adm (Bsh,pu), Node label}
bus_data = np.array([[1, 3, 0, 0, 101],
                     [2, 1, 0, 0, 102],
                     [3, 2, 0, 0, 103]])

# line_data format: {node 'from', node 'to', r_ij (pu), x_ij (pu), b_shunt_ij (pu), off-nominal tap ratio (pu), line limit (pu)}
line_data = np.array([[1, 2, 0, 0.1, 0, 0, 1.05],
                      [1, 3, 0, 0.25, 0, 0, 1.05],
                      [2, 3, 0, 0.2, 0, 0, 1.05]])

# load_data format: {bus ID, power factor, load in MW}
load_data= np.array([[2, 0.9799, 56.35]])

# gen_data format: {bus ID, Qmax (MVAR), Qmin (MVAR), Vreg (pu), generated real power (MW)}
gen_data = np.array([[3, 30, -25, 1.035, 52.82]])


In [10]:

line_data = pd.DataFrame(line_data).sort_values(by=[0,1]).values #line data sorting and data type change

bus_data = pd.DataFrame(bus_data).sort_values(by = [0]).values #Bus data sorting and data type change

load_data = pd.DataFrame(load_data).astype({0:'int'}).set_index(0) #Load data sorting and data type change

gen_data = pd.DataFrame(gen_data).astype({0:'int'}).set_index(0) #Generation data sorting and data type change

In [11]:
gen_data

,1,2,3,4
0,,,,
3,30.0,-25.0,1.035,52.82


In [12]:
base_mva = 100
R = line_data[:, 2]
X = line_data[:, 3]
B = 0.5j * line_data[:, 4]
taps = line_data[:,5]
Bsh = 1j* bus_data[:, 3]
Z = R + 1j * X
Y = 1 / Z
nline = len(line_data[:, 0])
nbus = int(np.amax(line_data[:, :2]))  # Total Number of buses

In [13]:
def tap_handle(x):
    if np.real(x) == 0:
        return 1
    else:
        return complex(x)
    
# Y Bus Formation
def Y_bus():
    Ybus = np.zeros((nbus, nbus), dtype=complex)
    Y1 = np.zeros((nbus, nbus), dtype=complex)
    np.fill_diagonal(Y1,Bsh)
    
    for k in range(nline):
        # Off Diagonal Elements
        Ybus[int(line_data[k, 0]) - 1, int(line_data[k, 1]) - 1] -= (Y[k])/np.conj(tap_handle(taps[k]))
        Ybus[int(line_data[k, 1]) - 1, int(line_data[k, 0]) - 1] -= (Y[k])/tap_handle(taps[k])

        # Diagonal Elements
        Ybus[int(line_data[k, 0]) - 1, int(line_data[k, 0]) - 1] += (Y[k] + B[k])/pow(abs(tap_handle(taps[k])),2)
        Ybus[int(line_data[k, 1]) - 1, int(line_data[k, 1]) - 1] += Y[k] + B[k]
    
    Y_bus_ = Ybus + Y1
    return Y_bus_

In [14]:
Y_bus()

array([[0.-14.j, 0.+10.j, 0. +4.j],
       [0.+10.j, 0.-15.j, 0. +5.j],
       [0. +4.j, 0. +5.j, 0. -9.j]])

In [132]:
#From Bus Data
bus_no = bus_data[:, 0].astype(int)
bus_type = bus_data[:,1].astype(int)

#From Load Data
pl = np.zeros(nbus)
pf = np.zeros(nbus)
delta = np.zeros(nbus)
for k in load_data.index:
    pl[k-1] = load_data.loc[k, 2]/base_mva #base power
    pf[k-1] = load_data.loc[k, 1]
ql = pl * np.tan(np.arccos(pf))

#From Generation Data
pg_ = np.zeros(nbus)
qg = np.zeros(nbus)
vmag = np.ones(nbus)
qmin = np.zeros(nbus)
qmax = np.zeros(nbus)
for k in gen_data.index:
    pg_[k-1] = (gen_data.loc[k, 4]).sum()/base_mva
    vmag[k-1] = np.mean(gen_data.loc[k, 3])
    qmin[k-1] = gen_data.loc[k, 2].sum()
    qmax[k-1] = gen_data.loc[k, 1].sum()


pg = pg_
b_data = pd.DataFrame()
b_data['Bus_No'] = bus_no
b_data['Bus_type'] = bus_type
b_data['Pg'] = pg
b_data['Qg'] = qg
b_data['Pl'] = pl
b_data['Ql'] = ql
b_data['Vmag'] = vmag
b_data['Delta'] = delta
b_data['Qmin'] = qmin/base_mva
b_data['Qmax'] = qmax/base_mva
b_data['Vr'] = vmag * np.cos(delta)
b_data['Vm'] = vmag * np.sin(delta)
b_data = b_data.set_index('Bus_No')
b_data['Psh'] = b_data['Pg'] - b_data['Pl']
b_data['Qsh'] = b_data['Qg'] - b_data['Ql']
b_data.head()

,Bus_type,Pg,Qg,Pl,Ql,Vmag,Delta,Qmin,Qmax,Vr,Vm,Psh,Qsh
Bus_No,,,,,,,,,,,,,
1,3,0.0000,0.0,0.0000,0.000000,1.000,0.0,0.00,0.0,1.000,0.0,0.0000,0.000000
2,1,0.0000,0.0,0.5635,0.114718,1.000,0.0,0.00,0.0,1.000,0.0,-0.5635,-0.114718
3,2,0.5282,0.0,0.0000,0.000000,1.035,0.0,-0.25,0.3,1.035,0.0,0.5282,0.000000


In [133]:
def I_r(Vr, Vm, P, Q):
    return (P*Vr + Q*Vm)/(pow(Vr, 2) + pow(Vm, 2))

def I_m(Vr, Vm, P, Q):
    return (P*Vm - Q*Vr)/(pow(Vr, 2) + pow(Vm, 2))

def F_set(Vr, Vm):
    return (pow(Vr, 2) + pow(Vm, 2))

def d_Ir_Vr(Vr, Vm, P, Q):
    V = pow(Vr,2) + pow(Vm,2)
    return (P* (pow(Vm, 2) - pow(Vr, 2)) - 2*Q*Vr*Vm)/pow(V,2)

def d_Ir_Vm(Vr, Vm, P, Q):
    V = pow(Vr,2) + pow(Vm,2)
    return (Q* (pow(Vr, 2) - pow(Vm, 2)) - 2*P*Vr*Vm)/pow(V,2)

def d_Im_Vr(Vr, Vm, P, Q):
    V = pow(Vr,2) + pow(Vm,2)
    return (Q* (pow(Vr, 2) - pow(Vm, 2)) - 2*P*Vr*Vm)/pow(V,2)

def d_Im_Vm(Vr, Vm, P, Q):
    V = pow(Vr,2) + pow(Vm,2)
    return (P* (pow(Vr, 2) - pow(Vm, 2)) - 2*Q*Vr*Vm)/pow(V,2)

def d_Ir_Q(Vr, Vm):
    V = pow(Vr,2) + pow(Vm,2)
    return Vm/pow(V,2)

def d_Im_Q(Vr, Vm):
    V = pow(Vr,2) + pow(Vm,2)
    return -Vr/pow(V,2)

def d_Fset_Vr(Vr):
    return 2*Vr

def d_Fset_Vm(Vm):
    return 2*Vm

In [134]:
'''J = np.array([[ G[0,0],  -B[0,0], -G[0,1],  B[0,1], -G[0,2],  B[0,2], 0, 0, 0],
              [ B[0,0],   G[0,0], -B[0,1], -G[0,1], -B[0,2], -G[0,2], 0, 0, 0],
              [-G[0,1],   B[0,1], G[1,1] + d_Ir_Vr(b_data.loc[2, 'Vr'], b_data.loc[2, 'Vm'], b_data.loc[2, 'Psh'], b_data.loc[2, 'Qsh']),  -B[1,1]+d_Ir_Vm(b_data.loc[2, 'Vr'], b_data.loc[2, 'Vm'], b_data.loc[2, 'Psh'], b_data.loc[2, 'Qsh']), -G[1,2],  B[1,2], 0, 0, 0],
              [-B[0,1],  -G[0,1], B[1,1] + d_Im_Vr(b_data.loc[2, 'Vr'], b_data.loc[2, 'Vm'], b_data.loc[2, 'Psh'], b_data.loc[2, 'Qsh']),  G[1,1] + d_Im_Vm(b_data.loc[2, 'Vr'], b_data.loc[2, 'Vm'], b_data.loc[2, 'Psh'], b_data.loc[2, 'Qsh']), -B[1,2], -G[1,2], 0, 0, 0],
              [-G[0,2], B[0,2], -G[1,2], B[1,2], G[2,2]+d_Ir_Vr(b_data.loc[3, 'Vr'], b_data.loc[3, 'Vm'], b_data.loc[3, 'Psh'], b_data.loc[3, 'Qsh']), -B[2,2] + d_Ir_Vm(b_data.loc[3, 'Vr'], b_data.loc[3, 'Vm'], b_data.loc[3, 'Psh'], b_data.loc[3, 'Qsh']), d_Ir_Q(b_data.loc[3, 'Vr'], b_data.loc[3, 'Vm']), 0, 0],
              [-B[0,2], -G[0,2], -B[1,2], -G[1,2], B[2,2]+d_Im_Vr(b_data.loc[3, 'Vr'], b_data.loc[3, 'Vm'], b_data.loc[3, 'Psh'], b_data.loc[3, 'Qsh']), G[2,2]+d_Im_Vm(b_data.loc[3, 'Vr'], b_data.loc[3, 'Vm'], b_data.loc[3, 'Psh'], b_data.loc[3, 'Qsh']),d_Im_Q(b_data.loc[3, 'Vr'], b_data.loc[3, 'Vm']), 0, 0 ],
              [0, 0, 0, 0, d_Fset_Vr(b_data.loc[3, 'Vr']), d_Fset_Vm(b_data.loc[3, 'Vm']), 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 1, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 1]])'''

"J = np.array([[ G[0,0],  -B[0,0], -G[0,1],  B[0,1], -G[0,2],  B[0,2], 0, 0, 0],\n              [ B[0,0],   G[0,0], -B[0,1], -G[0,1], -B[0,2], -G[0,2], 0, 0, 0],\n              [-G[0,1],   B[0,1], G[1,1] + d_Ir_Vr(b_data.loc[2, 'Vr'], b_data.loc[2, 'Vm'], b_data.loc[2, 'Psh'], b_data.loc[2, 'Qsh']),  -B[1,1]+d_Ir_Vm(b_data.loc[2, 'Vr'], b_data.loc[2, 'Vm'], b_data.loc[2, 'Psh'], b_data.loc[2, 'Qsh']), -G[1,2],  B[1,2], 0, 0, 0],\n              [-B[0,1],  -G[0,1], B[1,1] + d_Im_Vr(b_data.loc[2, 'Vr'], b_data.loc[2, 'Vm'], b_data.loc[2, 'Psh'], b_data.loc[2, 'Qsh']),  G[1,1] + d_Im_Vm(b_data.loc[2, 'Vr'], b_data.loc[2, 'Vm'], b_data.loc[2, 'Psh'], b_data.loc[2, 'Qsh']), -B[1,2], -G[1,2], 0, 0, 0],\n              [-G[0,2], B[0,2], -G[1,2], B[1,2], G[2,2]+d_Ir_Vr(b_data.loc[3, 'Vr'], b_data.loc[3, 'Vm'], b_data.loc[3, 'Psh'], b_data.loc[3, 'Qsh']), -B[2,2] + d_Ir_Vm(b_data.loc[3, 'Vr'], b_data.loc[3, 'Vm'], b_data.loc[3, 'Psh'], b_data.loc[3, 'Qsh']), d_Ir_Q(b_data.loc[3, 'Vr'], b_data.loc

In [135]:
def Jacobian(df):
    b_data = df.copy(deep=True)
    G = np.real(Y_bus())
    B = np.imag(Y_bus())

    J = np.array([[ G[0,0],  -B[0,0], -G[0,1],  B[0,1], -G[0,2],  B[0,2], 0, 0, 0],
                [ B[0,0],   G[0,0], -B[0,1], -G[0,1], -B[0,2], -G[0,2], 0, 0, 0],
                [-G[0,1],   B[0,1], G[1,1] + d_Ir_Vr(b_data.loc[2, 'Vr'], b_data.loc[2, 'Vm'], b_data.loc[2, 'Psh'], b_data.loc[2, 'Qsh']),  -B[1,1]+d_Ir_Vm(b_data.loc[2, 'Vr'], b_data.loc[2, 'Vm'], b_data.loc[2, 'Psh'], b_data.loc[2, 'Qsh']), -G[1,2],  B[1,2], 0, 0, 0],
                [-B[0,1],  -G[0,1], B[1,1] + d_Im_Vr(b_data.loc[2, 'Vr'], b_data.loc[2, 'Vm'], b_data.loc[2, 'Psh'], b_data.loc[2, 'Qsh']),  G[1,1] + d_Im_Vm(b_data.loc[2, 'Vr'], b_data.loc[2, 'Vm'], b_data.loc[2, 'Psh'], b_data.loc[2, 'Qsh']), -B[1,2], -G[1,2], 0, 0, 0],
                [-G[0,2], B[0,2], -G[1,2], B[1,2], G[2,2]+d_Ir_Vr(b_data.loc[3, 'Vr'], b_data.loc[3, 'Vm'], b_data.loc[3, 'Psh'], b_data.loc[3, 'Qsh']), -B[2,2] + d_Ir_Vm(b_data.loc[3, 'Vr'], b_data.loc[3, 'Vm'], b_data.loc[3, 'Psh'], b_data.loc[3, 'Qsh']), d_Ir_Q(b_data.loc[3, 'Vr'], b_data.loc[3, 'Vm']), 0, 0],
                [-B[0,2], -G[0,2], -B[1,2], -G[1,2], B[2,2]+d_Im_Vr(b_data.loc[3, 'Vr'], b_data.loc[3, 'Vm'], b_data.loc[3, 'Psh'], b_data.loc[3, 'Qsh']), G[2,2]+d_Im_Vm(b_data.loc[3, 'Vr'], b_data.loc[3, 'Vm'], b_data.loc[3, 'Psh'], b_data.loc[3, 'Qsh']),d_Im_Q(b_data.loc[3, 'Vr'], b_data.loc[3, 'Vm']), 0, 0 ],
                [0, 0, 0, 0, d_Fset_Vr(b_data.loc[3, 'Vr']), d_Fset_Vm(b_data.loc[3, 'Vm']), 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 1, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 1]])

    I_hist = np.array([[0, 0, d_Ir_Vr(b_data.loc[2, 'Vr'], b_data.loc[2, 'Vm'], b_data.loc[2, 'Psh'], b_data.loc[2, 'Qsh'])*b_data.loc[2, 'Vr'] + 
                        d_Ir_Vm(b_data.loc[2, 'Vr'], b_data.loc[2, 'Vm'], b_data.loc[2, 'Psh'], b_data.loc[2, 'Qsh'])*b_data.loc[2, 'Vm'] + 
                        -I_r(b_data.loc[2, 'Vr'], b_data.loc[2, 'Vm'], b_data.loc[2, 'Psh'], b_data.loc[2, 'Qsh']), 
                        d_Im_Vr(b_data.loc[2, 'Vr'], b_data.loc[2, 'Vm'], b_data.loc[2, 'Psh'], b_data.loc[2, 'Qsh'])*b_data.loc[2, 'Vr'] + 
                        d_Im_Vm(b_data.loc[2, 'Vr'], b_data.loc[2, 'Vm'], b_data.loc[2, 'Psh'], b_data.loc[2, 'Qsh'])*b_data.loc[2, 'Vm'] + 
                        -I_m(b_data.loc[2, 'Vr'], b_data.loc[2, 'Vm'], b_data.loc[2, 'Psh'], b_data.loc[2, 'Qsh']),
                        d_Ir_Vr(b_data.loc[3, 'Vr'], b_data.loc[3, 'Vm'], b_data.loc[3, 'Psh'], b_data.loc[3, 'Qsh'])*b_data.loc[3, 'Vr'] + 
                        d_Ir_Vm(b_data.loc[3, 'Vr'], b_data.loc[3, 'Vm'], b_data.loc[3, 'Psh'], b_data.loc[3, 'Qsh'])*b_data.loc[3, 'Vm'] + 
                        d_Ir_Q(b_data.loc[3, 'Vr'], b_data.loc[3, 'Vm'])-
                        I_r(b_data.loc[3, 'Vr'], b_data.loc[3, 'Vm'], b_data.loc[3, 'Psh'], b_data.loc[3, 'Qsh']),
                        d_Im_Vr(b_data.loc[3, 'Vr'], b_data.loc[3, 'Vm'], b_data.loc[3, 'Psh'], b_data.loc[3, 'Qsh'])*b_data.loc[3, 'Vr'] + 
                        d_Im_Vm(b_data.loc[3, 'Vr'], b_data.loc[3, 'Vm'], b_data.loc[3, 'Psh'], b_data.loc[3, 'Qsh'])*b_data.loc[3, 'Vm'] + 
                        d_Im_Q(b_data.loc[3, 'Vr'], b_data.loc[3, 'Vm'])-
                        I_m(b_data.loc[3, 'Vr'], b_data.loc[3, 'Vm'], b_data.loc[3, 'Psh'], b_data.loc[3, 'Qsh']),
                        b_data.loc[3, 'Vmag']- F_set(b_data.loc[3, 'Vr'], b_data.loc[3, 'Vm']) -d_Fset_Vr(b_data.loc[3, 'Vr']) - d_Fset_Vm(b_data.loc[3, 'Vm']),
                        b_data.loc[1, 'Vmag'] * np.cos(b_data.loc[1, 'Delta']),
                        b_data.loc[1, 'Vmag'] * np.sin(b_data.loc[1, 'Delta'])]]).T
    return I_hist, J

In [136]:
def N_R(df, iter, tol):
    "Function to perform Newton Raphson for solving non linear equations"
    b_data_ = df.copy(deep=True)
    for i in range(iter):
        I_hist, J = Jacobian(b_data_)
        DX = np.linalg.solve(J, I_hist) 
        print(DX)
        
        if max(np.abs(DX)) <= tol:
            return b_data_
        b_data_.loc[1, 'Vr'] = b_data_.loc[1, 'Vr'] + DX[0]
        b_data_.loc[1, 'Vm'] = b_data_.loc[1, 'Vm'] + DX[1]
        b_data_.loc[2, 'Vr'] = b_data_.loc[2, 'Vr'] + DX[2]
        b_data_.loc[2, 'Vm'] = b_data_.loc[2, 'Vm'] + DX[3]
        b_data_.loc[3, 'Vr'] = b_data_.loc[3, 'Vr'] + DX[4]
        b_data_.loc[3, 'Vm'] = b_data_.loc[3, 'Vm'] + DX[5]
        b_data_.loc[3, 'Qsh'] = b_data_.loc[3, 'Qsh'] + DX[6]
        b_data_.loc[2, 'Vm'] = b_data_.loc[2, 'Vm'] + DX[7]

    return "Value Error"
    

In [137]:
N_R(b_data, 10, 0.001)

[[-4.04985049e-01]
 [-1.16402496e-03]
 [ 4.97130818e-01]
 [-1.27161172e-03]
 [-1.01750000e+00]
 [ 1.05507088e-01]
 [ 1.48366179e+01]
 [ 1.00000000e+00]
 [ 0.00000000e+00]]
[[-0.70797187]
 [ 1.46899806]
 [ 0.89637344]
 [-1.8147298 ]
 [-1.94520321]
 [ 4.00745559]
 [ 5.87615421]
 [ 1.        ]
 [ 0.        ]]
[[-7.23408796e-02]
 [-1.14455308e+00]
 [ 7.25878136e-02]
 [ 1.41125714e+00]
 [-1.29910016e-01]
 [-2.97460473e+00]
 [ 2.68039029e+03]
 [ 1.00000000e+00]
 [ 0.00000000e+00]]
[[-5.18189925e-01]
 [-1.34925235e+00]
 [ 6.22761582e-01]
 [ 1.67370655e+00]
 [-1.33719861e+00]
 [-3.58366873e+00]
 [ 1.12535051e+04]
 [ 1.00000000e+00]
 [ 0.00000000e+00]]
[[-5.52341663e+00]
 [ 8.33421643e+00]
 [ 6.95266530e+00]
 [-1.02298065e+01]
 [-1.51383982e+01]
 [ 2.19957877e+01]
 [ 1.66883329e+06]
 [ 1.00000000e+00]
 [ 0.00000000e+00]]
[[-1.46623755e+03]
 [-1.34793746e+03]
 [ 1.79377072e+03]
 [ 1.68816196e+03]
 [-3.84375911e+03]
 [-3.66234264e+03]
 [ 2.26603763e+11]
 [ 1.00000000e+00]
 [ 0.00000000e+00]]
[[-3

'Value Error'